## 等時間線を求める

In [ ]:
%matplotlib inline
# モジュールのインポート
import pandas as pd
import urllib
import urllib.error
import urllib.request
import json
import numpy as np

import matplotlib.pyplot as plt


In [ ]:
def getdist(amode, orig, dest):
    # htmlの設定
    apikey = "********************" # Google APIkey を入れる
    html="https://maps.googleapis.com/maps/api/distancematrix/json?key={}".format(apikey)
    mode = "&mode={}".format(amode)
    origins="&origins={},{}".format(orig[0],orig[1])
    destinations = "&destinations={},{}".format(dest[0],dest[1])
    departure = "&departure_time=now"
    url = html+mode+origins+destinations+departure
    try:
        data = urllib.request.urlopen(url).read()
        return json.loads(data)
    except urllib.error.URLError as e:
        print(e)

def vec(r,theta,orig):
    # 出発点からの極座標表示
    dx = r*np.cos(theta*np.pi/180)
    dy = r*np.sin(theta*np.pi/180)
    x,y = orig
    return (x+dx,y+dy)


なるほど....
>In which countries are transit directions available?
The Google Maps Directions Service, which includes the Directions API and Distance Matrix API, supports all the transit providers in the transit coverage list, except for those **in Japan.**


In [ ]:
modes = ["walking", "driving","bicycling","transit"]
thetas = [i for i in range(0,360,10)]
# orig = (51.519763556894745, -0.12634440352170292) ## ロンドン
# orig = (35.68215955756656, 139.76700248367473) ## 東京駅
# orig = (50.130828242430816, 14.436294767800266) #プラハ
seconds =[86400, 10800,3600,1800,600] 
for m in seconds:
    result = {}
    for mode in modes:
        print("------{}-----".format(mode))
        xs = []
        ys = []
        for i in thetas:
            # 二部探索フェーズ
            r = 15
            l = 0.0001
            while r - l > 0.001:
                mid = (l*r)**0.5
                d = vec(mid, i, orig)
                ret = getdist(mode, orig, d)
                try: 
                    ans = ret["rows"][0]["elements"][0]
                    if "duration_in_traffic" in ans.keys():
                        s = ans["duration_in_traffic"]["value"]
                    else:
                        s = ans["duration"]["value"]
                except:
                    r = mid
                    continue
                if int(s) < m:
                    l = mid
                else:
                    r = mid
            print(i, r)
            # その角度でのradiusを出す
            cood = vec(r, i, orig)
            xs.append(cood[0])        
            ys.append(cood[1])
        result[mode]={"lat":xs, "lng":ys}
    with open('data/data_{}.json'.format(m), 'w') as f:
        json.dump(result, f, indent=4)
